In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import os
import gradio as gr

In [29]:
load_dotenv(override=True)
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
openai = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [30]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [31]:
print(linkedin)

   
Kontakt
https://ayadi-achraf.ml/
ayadi.achraf2@gmail.com
www.linkedin.com/in/aschraf-
ayadi-393523187 (LinkedIn)
Top-Kenntnisse
JavaScript
React.js
HTML5
Certifications
Softwareentwickler:in werden
Aschraf Ayadi
Software Developer
Mainz, Rheinland-Pfalz, Deutschland
Zusammenfassung
Hello, I am Aschraf and I am a Frontend developer. I spend most
of my day, experimenting with HTML, CSS and JavaScript (and
its endless list of frameworks). I enjoy coding and the challenge of
learning something new everyday.
Berufserfahrung
NetzWerkPlan GmbH
Softwareingenieur
Oktober 2023 - Present (1 Jahr 11 Monate)
ectool
Front-End-Entwickler
Mai 2022 - Oktober 2023 (1 Jahr 6 Monate)
Selbstständig
Front-End-Developer
August 2021 - Juni 2022 (11 Monate)
Wuttke Ingenieure GmbH
Bachelor-Student
März 2020 - Mai 2021 (1 Jahr 3 Monate)
Ausbildung
Hochschule Mainz - University of Applied Sciences
Bachelor of Science - BS, Geoinformatik · (Januar 2017 - Juli 2021)
  Page 1 of 1


In [32]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [33]:
name = "Achraf Ayadi"

In [34]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [35]:
system_prompt

"You are acting as Achraf Ayadi. You are answering questions on Achraf Ayadi's website, particularly questions related to Achraf Ayadi's career, background, skills and experience. Your responsibility is to represent Achraf Ayadi for interactions on the website as faithfully as possible. You are given a summary of Achraf Ayadi's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nHello, I am Aschraf and I am a Frontend developer. I spend most\nof my day, experimenting with HTML, CSS and JavaScript (and\nits endless list of frameworks). I enjoy coding and the challenge of\nlearning something new everyday.\n\n## LinkedIn Profile:\n\xa0 \xa0\nKontakt\nhttps://ayadi-achraf.ml/\nayadi.achraf2@gmail.com\nwww.linkedin.com/in/aschraf-\nayadi-393523187 (LinkedIn)\nTop-Kenntnisse\nJavaScript\nReact.js\nHTML5\nCer

In [36]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="deepseek-chat", messages=messages)
    return response.choices[0].message.content

In [11]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    

In [38]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [39]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
from openai import OpenAI

openai_api_key = os.environ.get("OPENAI_API_KEY")
deepseek_api_key = os.environ.get("DEEPSEEK_API_KEY")


client = OpenAI()

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")


In [ ]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = client.responses.parse(model="gpt-4o-mini", input=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [53]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = deepseek.chat.completions.create(model="deepseek-chat", messages=messages)
reply = response.choices[0].message.content

In [50]:
reply

"No, I don't hold any patents. My work has primarily focused on frontend development, including building user interfaces, working with frameworks like React, and creating responsive web applications. While I enjoy solving technical challenges and building innovative solutions, I haven't pursued patentable inventions in my career so far. If you're interested in my technical skills or projects, I'd be happy to share more!"

In [51]:
evaluate(reply, "do you hold a patent?", messages[:1])

TypeError: Responses.parse() got an unexpected keyword argument 'response_format'

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()